# Predict one charazter

In [ ]:
import googleapiclient.discovery
import sys
import numpy as np
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"
# Load values
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# Clean the text
input_clean=clean_text(input_init.lower())
input_clean = input_clean[:window_size]
# Text to array [1,input_lenght,num_chars]
x_test = np.zeros((window_size, number_chars))
for t, char in enumerate(input_clean):
    if char in chars_to_indices:
        x_test[ t, chars_to_indices[char]] = 1.
#x_test   
#x_test = np.zeros((number_chars,window_size))
#for t, char in enumerate(input_clean):
#    if char in chars_to_indices:
#        x_test[ chars_to_indices[char],t] = 1.
print(len(x_test[0]))    
project="ai-ml-dl"
model="BeirasRnn"
version="v10"
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(project, model)
if version is not None:
        name += '/versions/{}'.format(version)
instances={'sequence':x_test.tolist()}
response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
if 'error' in response:
    raise RuntimeError(response['error'])
test_predict=np.array(response['predictions'][0]['scores'])
r = np.argmax(test_predict)  # predict class of each test input
print(test_predict)
print(r,indices_to_chars[r])

# Predict sentece

In [7]:
# Define a function
def predict_one(text_predict,service,model_name,window_size,chars_to_indices, indices_to_chars):
    # Convert input sequence to array
    number_chars=len(chars_to_indices)
    x_test = np.zeros((window_size,number_chars))
    for t, char in enumerate(text_predict):
       x_test[t,chars_to_indices[char]] = 1.
    #print(x_test.shape)
    x_test=x_test[:window_size,:]
    #Prepare the request
    instances={'sequence':x_test.tolist()}
    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
    if 'error' in response:
        raise RuntimeError(response['error'])
    test_predict=np.array(response['predictions'][0]['scores'])
    r = np.argmax(test_predict)  # predict class of each test input
    return (indices_to_chars[r])

In [8]:
# Complete a sequence using the server
def predict_window(text_predict,number_predict,window_size,lproject,lmodel,lversion):
    
    # Get dictionaries
    chars_to_indices, indices_to_chars = load_coded_dictionaries()
    # Clean the test
    input_clean=clean_text(text_predict.lower())
    # Get stub
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(lproject, lmodel)
    if lversion is not None:
        name += '/versions/{}'.format(lversion)
    print(name)
    # Call server for all charazters
    for i in range(number_predict):
        d=predict_one(input_clean[i:],service,name,window_size,chars_to_indices, indices_to_chars)
        input_clean+=d
    return input_clean

In [10]:
import googleapiclient.discovery
project="ai-ml-dl"
model="BeirasRnn"
version="v10"

input_init=u"se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"

window_size = 100
number_predict=200
      

response=predict_window(input_init[:window_size],number_predict,
                        window_size,project,model,version)
print(response)

projects/ai-ml-dl/models/BeirasRnn/versions/v10
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcumalo de estado español e o proceso de descomposición do poder constitucional e desembocar nestas asembleas a unha concepción do sistema de estado e o proceso de descomposición do sistema-mundo as cond
